In [1]:
import pandas as pd
import swisseph as swe
from vedicastro.utils import *
from vedicastro import VedicAstro as va

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
import os 

current_dir = os.path.abspath(os.curdir)
subfolder_path = os.path.join(current_dir, 'swefiles')

swe.set_ephe_path(subfolder_path)

In [4]:
def decimal_to_dms(decimal_deg: float):
    zodiac_signs = ['Aries', 'Taurus', 'Gemini', 'Cancer', 'Leo', 'Virgo', 'Libra', 'Scorpio', 'Sagittarius', 'Capricorn', 'Aquarius', 'Pisces']
    deg, mins, sec, fractional_sec, sign_idx = swe.split_deg(decimal_deg, roundflag = swe.SPLIT_DEG_ZODIACAL )
    sec += fractional_sec  # Add fractional seconds to seconds
    return zodiac_signs[sign_idx], f"{deg:02d}:{mins:02d}:{int(sec):02d}"

def get_rl_nl_sl_data(deg : float):
    """
    Returns the  Rashi (Sign) Lord, Nakshatra, Nakshatra Pada, Nakshatra Lord, Sub Lord and Sub Sub Lord 
    corresponding to the given degree.
    """
    from vedicastro.VedicAstro import NAKSHATRAS, SIGN_LORDS
    duration = [7, 20, 6, 10, 7, 18, 16, 19, 17]

    lords = ["Ketu", "Venus", "Sun", "Moon", "Mars", "Rahu", "Jupiter", "Saturn", "Mercury"]

    star_lords = lords * 3 ## lords for the 27 Nakshatras

    ## Compute Sign lords
    sign_deg = deg % 360  # Normalize degree to [0, 360)
    sign_index = int(sign_deg // 30)  # Each zodiac sign is 30 degrees
    
    # Compute Nakshatra details
    nakshatra_deg = sign_deg % 13.332  # Each nakshatra is 13.332 degrees
    nakshatra_index = int(sign_deg // 13.332)  # Find the nakshatra index
    pada = int((nakshatra_deg % 13.332) // 3.325) + 1  # Each pada is 3.325 degrees

    # Ensure nakshatra_index is within bounds
    nakshatra_index = nakshatra_index % len(NAKSHATRAS)        

    # Compute SubLords
    deg = deg - 120 * int(deg / 120)
    degcum = 0
    i = 0

    while i < 9:
        deg_nl = 360 / 27
        j = i
        while True:
            deg_sl = deg_nl * duration[j] / 120
            k = j
            while True:
                deg_ss = deg_sl * duration[k] / 120
                degcum += deg_ss
                if degcum >= deg:
                    return {"Nakshatra": NAKSHATRAS[nakshatra_index] + "-" + str(pada), 
                            "RasiLord": SIGN_LORDS[sign_index], "NakshatraLord": star_lords[nakshatra_index],
                            "SubLord": lords[j], "SubSubLord": lords[k] }
                k = (k + 1) % 9
                if k == j:
                    break
            j = (j + 1) % 9
            if j == i:
                break
        i += 1  

### Common `SideReal` Ayanamsas for Vedic Astrology
- https://astrorigin.com/pyswisseph/sphinx/ephemerides/sidereal/krishnamurti_ayanamshas.html
- https://astrorigin.com/pyswisseph/sphinx/ephemerides/sidereal/spica_citra_tradition_and_lahiri_ayanamsha.html


In [5]:
# SWE_AYANAMSHAS = { "Krishnamurti_Old" : swe.SIDM_KRISHNAMURTI, "Krishnamurti_New": swe.SIDM_KRISHNAMURTI_VP291,
#                 "Lahiri_1940": swe.SIDM_LAHIRI_1940, "Lahiri_VP285" : swe.SIDM_LAHIRI_VP285, 
#                 "Lahiri_ICRC" : swe.SIDM_LAHIRI_ICRC,
#                 }
SWE_AYANAMSHAS = {  "Krishnamurti_New": swe.SIDM_KRISHNAMURTI_VP291
                }

In [6]:
## Case 1
# year = 2024
# month = 8
# day = 14
# hour = 22
# minute = 31
# secs = 31
# latitude, longitude, utc = 28.6334, 77.2834, "+5:30" ## Delhi

In [7]:
## Case 2
year = 2000
month = 12
day = 30
hour = 22
minute = 31
secs = 59
latitude, longitude, utc_offset = 28.6334, 77.2834, "+5:30" ## Delhi

In [8]:
from pytz import timezone
zone = "Asia/Calcutta"
### TIMEZONE INFORMATION
tz = timezone(zone)
localized_time = tz.localize(datetime(year, month, day, hour, minute, secs))

### `Asc` Results using native `pyswisseph` library


In [9]:
utc_float =  utc_offset_str_to_float(utc_offset)

utc = swe.utc_time_zone(year, month, day, hour = hour, minutes = minute, seconds = secs, offset = utc_float)
_ , jd_start = swe.utc_to_jd(*utc) ## Unpacks utc tuple
current_time = jd_start

In [10]:
# Assuming SWE_AYANAMSHAS and other necessary imports and variables are already defined

print(swe.get_current_file_data(0)) # prints the name of current ephemeris file being used
results = []  # List to store the results

for ayan in list(SWE_AYANAMSHAS.keys()):
    swe.set_sid_mode(SWE_AYANAMSHAS.get(ayan))  # Set the ayanamsa based on the current key
    cusps, asmc = swe.houses_ex(current_time, latitude, longitude, b'P', flags=swe.FLG_SIDEREAL)
    ## Compute results for only Ascendant
    # asc = cusps[0]
    # sign, sign_dms = decimal_to_dms(asc)
    # res = { 'Ayanamsa': ayan, 'AscSign': sign, 'AscDecLong': asc, 'AscDMS': sign_dms}
    # res.update(get_rl_nl_sl_data(deg = asc))
    # results.append(res)

    ## Compute data for all cusps
    for idx, cusp in zip(range(1, len(cusps)+1),cusps):
        sign, sign_dms = decimal_to_dms(cusp)
        # res = {'Ayanamsa': ayan, "HouseID" : idx,'DecLong': cusp, 'Sign': sign,'SignDMS': sign_dms}
        res = {"HouseID" : idx,'DecLong': cusp, 'Sign': sign,'SignDMS': sign_dms}
        res.update(get_rl_nl_sl_data(deg = cusp))
        results.append(res)


# Convert the list of dictionaries into a pandas DataFrame
df_swisseph = pd.DataFrame(results)

('c:\\Users\\acer\\Desktop\\VedicAstro\\test_suite\\swefiles\\sepl_18.se1', 0.0, 0.0, 0)


In [12]:
# Assuming SWE_AYANAMSHAS and other necessary imports and variables are already defined

swe.set_jpl_file("de405.eph")
print(swe.get_current_file_data(0)) # prints the name of current ephemeris file being used

results = []  # List to store the results
for ayan in list(SWE_AYANAMSHAS.keys()):
    swe.set_sid_mode(SWE_AYANAMSHAS.get(ayan))  # Set the ayanamsa based on the current key
    cusps, asmc = swe.houses_ex(current_time, latitude, longitude, b'P', flags=swe.FLG_SIDEREAL)
    ## Compute results for only Ascendant
    # asc = cusps[0]
    # sign, sign_dms = decimal_to_dms(asc)
    # res = { 'Ayanamsa': ayan, 'AscSign': sign, 'AscDecLong': asc, 'AscDMS': sign_dms}
    # res.update(get_rl_nl_sl_data(deg = asc))
    # results.append(res)

    ## Compute data for all cusps
    for idx, cusp in zip(range(1, len(cusps)+1),cusps):
        sign, sign_dms = decimal_to_dms(cusp)
        # res = {'Ayanamsa': ayan, "HouseID" : idx,'DecLong': cusp, 'Sign': sign,'SignDMS': sign_dms}
        res = {"HouseID" : idx,'DecLong': cusp, 'Sign': sign,'SignDMS': sign_dms}
        res.update(get_rl_nl_sl_data(deg = cusp))
        results.append(res)


# Convert the list of dictionaries into a pandas DataFrame
df_swisseph2 = pd.DataFrame(results)

('c:\\Users\\acer\\Desktop\\VedicAstro\\test_suite\\swefiles\\de405.eph', 0.0, 0.0, 0)


In [13]:
# Convert DataFrames to HTML and concatenate them with some space in between
df1_html = df_swisseph.to_html()
df2_html = df_swisseph2.to_html()

# df1_html = df_swisseph[df_swisseph["Ayanamsa"] == "Krishnamurti_New"].to_html()
# df2_html = df_flatlib[df_flatlib["Ayanamsa"] == "Krishnamurti"].to_html()

# Titles for each DataFrame
title1 = "Swisseph - (large)"
title2 = "Swisseph - (lite)"

# Display them side by side with titles using HTML display with formatted string
html = f"""
<b>Housewise Results for :</b> {localized_time} at <b>Lat:</b>{latitude} | <b>Long:</b> {longitude}
<div style='display:flex; justify-content:space-between;'>
    <div >
        <h2>{title1}</h2>  <!-- Title for the first DataFrame -->
        {df1_html}
    </div>
    <div >
        <h2>{title2}</h2>  <!-- Title for the second DataFrame -->
        {df2_html}
    </div>
</div>
"""
display(HTML(html))

,HouseID,DecLong,Sign,SignDMS,Nakshatra,RasiLord,NakshatraLord,SubLord,SubSubLord
0,1,140.570036,Leo,20:34:12,PūrvaPhalgunī-3,Sun,Venus,Jupiter,Saturn
1,2,167.696585,Virgo,17:41:47,Hasta-3,Mercury,Moon,Saturn,Jupiter
2,3,197.988661,Libra,17:59:19,Svati-4,Venus,Rahu,Sun,Ketu
3,4,229.813822,Scorpio,19:48:49,Jyeshtha-1,Mars,Mercury,Venus,Sun
4,5,261.497059,Sagittarius,21:29:49,PurvaAshadha-3,Jupiter,Venus,Jupiter,Moon
5,6,292.012698,Capricorn,22:00:45,Shravana-4,Saturn,Moon,Venus,Saturn
6,7,320.570036,Aquarius,20:34:12,PurvaBhādrapadā-1,Saturn,Jupiter,Jupiter,Mercury
7,8,347.696585,Pisces,17:41:47,Revati-1,Jupiter,Mercury,Mercury,Mars
8,9,17.988661,Aries,17:59:19,Bharani-2,Mars,Venus,Mars,Venus
9,10,49.813822,Taurus,19:48:49,Rohini-3,Venus,Moon,Ketu,Venus
